## Multiclass Classification using Keras and TensorFlow on Food-101 Dataset

### Overview


   * Download and extract Food 101 dataset
   * Understand dataset structure and files
   * Visualize random image from each of the 101 classes
   * Split the image data into train and test using train.txt and test.txt
   * Create a subset of data with few classes(3) - train_mini and test_mini for experimenting
   * Fine tune Inception Pretrained model using Food 101 dataset
   * Visualize accuracy and loss plots
   * Predicting classes for new images from internet
   * Scale up and fine tune Inceptionv3 model with 11 classes of data
   * Model Explainability
   * Summary of the things I tried
   * Further improvements
   * Feedback


In [1]:
import tensorflow as tf
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import keras.backend as K
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from keras import regularizers
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.optimizers import SGD
from keras.regularizers import l2
from tensorflow import keras
from keras import models
import cv2

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Check if GPU is enabled

In [2]:
# Check if GPU is enabled
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

1.8.0
/device:GPU:0


In [4]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


### Download and Extract Food 101 Dataset

In [5]:
# Helper function to download data and extract
def get_data_extract():
    if "food_101" in os.listdir("."):
        print("Dataset already exists")
    else:
        print("Downloading the data...")
        !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
        print("Dataset downloaded")
        print("Extracting data..")
        !tar xzvf food-101.tar.gz | head -10
        print("Extraction done!")

In [7]:
get_data_extract() # showing only the first few lines of extracting

--2020-04-04 07:40:47--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2020-04-04 07:40:48--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: 'food-101.tar.gz.1'

food-101.tar.gz.1   100%[===================>]   4.65G  12.7MB/s    in 6m 34s  

2020-04-04 07:47:23 (12.1 MB/s) - 'food-101.tar.gz.1' saved [4996278331/4996278331]

Dataset downloaded
Extracting data..
food-101/
food-101/images/
food-101/images/beignets/
food-101/images/beignets/2885220.jpg
food-101/images/beignets/1814040.jpg
foo

### Understand dataset structure and files

In [ ]:
# Check the extracted dataset folder
!ls food-101